# **Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación.**
## Base de datos: Diabetes

En primer lugar, cargamos la base de datos guardada como un fichero .csv y cargamos las principales librerias de python.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Representacion de datos
import seaborn as sns
import matplotlib.pyplot as plt

Importamos el resto de librerias necesarias para realizar el análisis y un script que nos ayudará a mostrar este cuaderno de una forma más ordenada sin la necesidad de añadir funciones dentro de este.

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

Añadimos una semilla para que los procesos creados en el estudio sean repetibles y reproducibles. Vamos a utilizar la misma semilla que en el estudio "iris".

In [ ]:
seed = 12737

## Acceso y almacenamiento de datos.

Cargamos el conjunto de datos de diabetes, a través de la libreria de pandas.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
target = 'Outcome'

data = utils.load_data(filepath, None, target)

Vamos a imprimir por pantalla 5 ejemplos de nuestra base de datos de forma aleatoria, para que nuestra muestra imprimida no esté `sesgada` y ver como se estructura la información.

In [ ]:
data.sample(5,random_state=seed)

Nuestra base de datos tiene 9 columnas o variables:

### Variables predictoras
* `Pregnancies (Embarazos)`

* `Glucose (Glucosa)`

* `BloodPressure (Presión de sangre)`

* `SkinThickness (Espesor de la piel)`

* `Insulin (Insulina)`

* `BMI (IMC)`

* `DiabetesPedigreeFunction (Función de pedigrí de la diabetes)`

* `Age (Edad)`

### Variable clase
* `Outcome (Resultado)`


Dividimos la base de datos en las variables predictoras, guardadas en el atributo `X`. Y la variable clase en el atributo `y`.

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Vamos a comprobar que la división de la base de datos se ha realizado de forma correcta y tiene todas las variables predictoras.

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.cat.categories

Vamos a separar el conjunto de datos en dos conjuntos diferentes: `Entrenamiento` y `test`.

Este proceso llamado `houldout`, lo utilizamos para no sobreajustar el modelo de entrenamiento y validar de una forma más honesta los resultados dados por el modelo.

* La muestra de entrenamiento será el 70% de la base de datos.
* La muestra de test será el 30% de la base de datos.

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
X_train.sample(5, random_state=seed)

## Análisis exploratorio de datos.

Con el siguiente codigo, vamos a mostar la información de todas las variables de nuestra base de datos para ayudarnos en nuestro estudio.


In [ ]:
data.info(memory_usage=False)

La variable clase `Outcome` en este estudio es una variable numérica entera, discreta con dos posibles valores: 0 y 1. Este valor indica si dada la instancia de la base de datos, el resultado de su diagnostico es positivo en diabetes.

El resto de variables, las predictoras, tambien son numéricas de tipo entero. Excepto `BMI` y `DiabetePedigreeFunction`, que son de tipo flotante.

## Visualización de los datos.

A partir de la información que ya conocemos de las variables, vamos a representar los datos en de diferentes formas. Esto lo haremos para realizar un análisis en profundidad.

Primero vamos a utilizar métodos `univariados`. Al analizar las variables por separados podemos estudiar si alguna de estas variables contiene datos ruidosos o `outliers`. También podemos observar de esta manera la distribución de valores de una variable, si una variable es uniforme no nos aportará mucha información ya que tendrá siempre el mismo valor. Mientras que, si nuestra distribución es gaussiana nos ayudará a la hora de realizar el estudio.

Antes de nada, las variables de entrenamiento las unimos para tener una base de datos de entrenamiento completa.

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

A partir de un histograma podemos ver la densidad de ejemplos para distintos valores de una variable numérica. Tambíen nos permite conocer información de las variables por separado como hemos dicho anteriormente. La siguiente gráfica interactiva permite ver la distribución de cada variable por separado, pudiendo ver a simple vista valores ruidosos o la distribución de cada variable. En este caso, la gráfica lo utilizaremos para observar `outliers`.

In [ ]:
utils.plot_histogram(data_train)

Podemos observar en las gráficas como hay variables que poseen valores nulos, en este caso estos valores nulos estan representados por un valor `0`. Ya que, muchos atributos no tienen sentido porque no se puede tener este valor, pero hay algunos atributos que tener valores a 0 si tienen sentido. Lo estudiaremos a continuación.

Vamos a estudiar los atributos y si tienen *outliers* por orden que aparecen en la gráfica.

`Pregnancies`: Tiene algunso ejemplos en la base de datos que se aleja de la distribución de valores para este atributo. Esta variable al indicar los embarazos, tener valores tan altos no tiene sentido, lo cual manejaremos en el preprocesamiento.

`Glucose`: Este atributo poseé dos ejemplos con valores a 0. Lo cuál es un valor erroneo, ya que se necesita una medida. Este valor es un valor nulo para nuestra base de datos. Estos valores nulos los controlaremos en el preprocesamiento.

`BloodPressure`: Al igual que en Glucose también tiene valores a 0, a su vez tambíen hay muestras de datos que están alejados de la distribución normal de la variable.

`SkinThickness`: Posee muchos ejemplos en la base de datos con un valor a 0 o nulo. Esto supondrá un problema para el score que nos darán los modelos. En estos casos si el `20%` de los valores del atributo son nulos, eliminaremos la variable completa, ya que no introduce conocimiento adicional.

`Insulin`: Tienen muchos *outliers* que se alejan mucho de la distribución, también posee una gran cantidad de valores nulos. Es la variable con más valores a 0. Al igual que el anterior atributo, estudiaremos si la variable puede ser eliminada y en caso de que sea así lo haremos en el preprocesamiento.

`BMI`: Otro atributo que posee valores nulos que tendremos que manejar en el preprocesamiento.

`DiabetesPedigreeFunction`: Este atributo es similar a Insulin, tiene *outliers* muy alejados de la media de la distribución del atributo. Con unos pocos ejemplos de valores nulos.

`Age`: Esta variable no supone muchos problemas con los *outliers*, teniendo muy pocos ejemplos de estos en la base de datos.



Como hemos comentado anteriormente, hay variables con gran cantidad de valores nulos. Vamos a dar el porcentaje que supone estos valores y si es `mayor que el 20%`, eliminaremos completamente todo el atributo. Estas variables son `SkinThickness` e `Insulin`. 

In [ ]:
valor_nulo = 0
data_train['SkinThickness'].value_counts(normalize = True)[valor_nulo] * 100

Para `SkinThickness` 29.24% de los valores del atributo son valores nulos, a 0. Entonces, esta variable la quitaremos de nuestra base de datos.

In [ ]:
data_train['Insulin'].value_counts(normalize = True)[valor_nulo] * 100

Para `Insulin` el 48.6% de los valores son nulos, por tanto esta variable contiene una gran cantidad de estos valores y será eliminada.

In [ ]:
data_train['BloodPressure'].value_counts(normalize = True)[valor_nulo] * 100

Por último, para la variable `BloodPressure` el 4.66% son valores nulos. Por tanto esta variable no será eliminada y realizaremos una imputación para dar un valor correcto a estos valores nulos.

A continuación, vamos a estudiar la distribución de cada atributo. Mediante la libreria `seaborn` y la función `displot` podemos conocer cada distribución de una forma gráfica y ver que atributos se acercan a una distribución gaussiana o normal.

Con una función creada por nosotros, llamada `plot_distribution` y guardada en el script de `utils`, representaremos gráficamente la distribución de todas las variables para estudiarla de una forma más comoda. Ignorando los valores nulos y ruidosos que hemos visto en el anterior apartado.

In [ ]:
def plot_distribution(data):
    
    #Busca el número de columnas mayor para que las gráficas se representen como una matriz
    col = 0
    for i in range(1,10):
        if len(data.columns)%i == 0 and i != len(data.columns):
            col = i
    
    x = (len(data.columns)//col)
    if col == 1:
        x,col = 1,len(data.columns) 
    fig, axes = plt.subplots(nrows=x, ncols=col, figsize=(15,15))
    k = 0
    print()
    for i in range(0,x):
        for j in range(0,col):
            column = list(data)[k]
            k+=1
            sns.distplot(data[column],ax=axes[i,j])
        

In [ ]:
plot_distribution(X_train)

Aunque en las gráficas salgan las variables Insulin y SkinThickness, no las estudiaremos ya que estos atributos los elimaneremos de nuestra base de datos como hemos comentado anteriormente.

En las gráficas podemos observar distribuciones gaussianas o normales en forma de campana, estos atributos son: `BloodPressure`, `Glucose` y `BMI`.

El resto de atributos se asemejan también a una campana pero tienen valores que hacen que esa distribución cambie. Quitando `outliers` como en la variable `Pregnancies`, tendremos una distribución más gaussiana. Entonces, para este tipo de atributos necesitaremos un método para quitar estos valores atípicos.

Por último, para el análisis univariado, vamos a comprobar el número de casos diferentes para la variable clase. Comprobando si nuestra base de datos de entrenamiento esta `balanceada`.

In [ ]:
utils.plot_barplot(data_train)

In [ ]:
print("Valores:\n",data_train['Outcome'].value_counts(), "\n\nFrecuencia:\n",data_train['Outcome'].value_counts(normalize = True)*100)

El número de ejemplos en nuestra base de datos de entrenamiento esta `desbalanceado`. El número de resultados que son `0` es de **350 ejemplos**, mientras que de resultado `1` es de **187 ejemplos**. Esto supone que el `65.18%` de nuestra variable clase, es de valor `0`.

In [ ]:
data_train.describe(include="number")

Estás estadísticas una vez realizado el preprocesamiento variarán, al quitar valores que hemos estudiado que resultarán un problema para la discretización y la creación de buenos modelos.

Una vez estudiadas las variables de forma individual, para completar el estudio de las variables, tenemos que ver las relaciones entre estas variables. Este estudio llamado `multievaluado`, nos muestra información imporante sobre el discretizado de las variables y su potencia discriminatoria. Tambíen, sobre los valores ruidosos que se pueden encontrar en una zona con gran cantidad de otros valores clase.

In [ ]:
#utils.plot_pairplot(data_train.iloc[:,0:3],target="Outcome")

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.iloc[:,0:3], y_train), target='Outcome')

In [ ]:
utils.plot_pairplot(utils.join_dataset(X_train.iloc[:,5:9], y_train), target='Outcome')

Estás gráficas nos dan una representación de los valores de `Outcome` para cada par de variables. Podemos sacar un valor de corte y las variables necesarias para discretizar una variable clase y tener un modelo correcto. Pero, al tener tantas variables la claridad de sacar a simple vista un resultado correcto se vuelve muy difícil.

## Preprocesamiento de datos.

El preprocesamiento de datos es la tarea más importante del proceso KDD. En este apartado realizaremos:

* `Limpieza de datos`: Suavizaremos el ruido y eliminaremos datos problematicos
* `Integración de datos`: Introduciremos datos donde sean nulos, se utilizará una imputación de los valores perdidos mediante un estimador.
* `Transformación de datos`: Normalizaremos los datos.
* `Reducción de datos`: Discretizaremos los datos si es necesario.

Para esta tarea se utilizara un `pipeline` para no cometer una `fuga de datos` y no introducir datos del conjunto de entrenamiento donde se aprenderá el modelo en el conjunto de prueba donde los datos de este conjunto son `datos crudos`. Una vez elaborado este apartado se tendrá que validad el modelo entrenado.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import QuantileTransformer

from sklearn.preprocessing import StandardScaler

In [ ]:
delete_colum = ['SkinThickness','Insulin']
def drop_column(X, columns = delete_colum):
    return X.drop(columns, axis=1)
#X_test = drop_column(X_test,delete_colum)
#X_train = drop_column(X_train,delete_colum)

In [ ]:
#X_train = X_train.drop(columns = ['SkinThickness','Insulin'])
#X_test = X_test.drop(columns = ['SkinThickness','Insulin'])

In [ ]:
quantile_transformer = preprocessing.QuantileTransformer(n_quantiles = 10,output_distribution='normal',random_state=seed)

In [ ]:
imputer_col = list(set(list(X)) - set(delete_colum) - set(['Pregnancies']))
print(imputer_col)

In [ ]:
imputer = make_column_transformer((KNNImputer(n_neighbors=3, weights="uniform",missing_values=0), imputer_col))

In [ ]:
#plot_distribution(X_imputer)

## Algoritmos de clasificación.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="uniform")

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

In [ ]:
discretize_tree_model = make_pipeline(discretizer, tree_model)

In [ ]:
pip = make_pipeline(FunctionTransformer(drop_column),imputer,discretizer, tree_model)

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.evaluate(pip,
               X_train, X_test,
               y_train, y_test)